# 0. Data

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.linalg import block_diag # for computation efficiency - Kronecker product with identity matrix and regular matrix

In [37]:
'split data'
'first 21 columns are input variables and last 7 columns are output variables'

train_data = pd.read_csv('./data/SARCOSTst.csv', header=None)[:1000]
test_data = pd.read_csv('./data/SARCOSTrn.csv', header=None)[:100]

X_train, X_valid, Y_train, Y_valid = train_test_split(train_data.iloc[:,:21], train_data.iloc[:,-7:], test_size=0.2)
X_test, Y_test = test_data.iloc[:,:21], test_data.iloc[:,-7:]

print('train set: ', X_train.shape, Y_train.shape)
print('valid set: ', X_valid.shape, Y_valid.shape)
print('test set: ', X_test.shape, Y_test.shape)

train set:  (800, 21) (800, 7)
valid set:  (200, 21) (200, 7)
test set:  (100, 21) (100, 7)


# 1. Model

## Gaussian kernel

In [38]:
class GaussianKernel:
    """
    isotropic Gaussian kernel
    :input: X1 (N*D), X2 (M*D)
    :output: covariance matrix (N*M)
    """
    def __init__(self, sigma_k=1):
        self.sigma_k = sigma_k # isotropic Gaussian kernel variance (Hyperparameter !)

    def __call__(self, X1, X2):
        return np.exp(-(np.sum(X1**2, axis=1).values.reshape(-1, 1) +
                        np.sum(X2**2, axis=1).values.reshape(1, -1) - 2*X1@X2.T) / pow(self.sigma_k, 2))


## Linear kernel
- Standard dot product kernel
    - k(a,b) = a^\top b
- This is the same as Bayesian linear regression model without using a nonlinear feature map

In [33]:
class LinearKernel: # X_train, X_test -> 4분
    """
    standard dot product kernel
    :input: X1 (N*D), X2 (M*D)
    :output: covariance matrix (N*M)
    """

    def __init__(self):
        pass

    def __call__(self, X1, X2):
        return np.array([ [np.dot(x1, x2) for x1 in np.array(X1)] for x2 in np.array(X2) ])


## GP regression

In [34]:
class GaussianProcessRegression:
    def __init__(self, X_train, Y_train, K=GaussianKernel(), sigma_n=1):
        self.X_train = X_train
        self.Y_train = Y_train
        self.K = K
        self.sigma_n = sigma_n  # noise variance (Hyperparameter !)
        self.predictive_distribution = None
        self.temp = None

    def predict(self, X_test):
        
        # sufficient statistics

        D = self.Y_train.shape[1]
        C = np.identity(D) # coregionalisation matrix (used for multi-output model) # 7x7

        K_X_X = self.K(self.X_train, self.X_train)  # 31587x31587
        K_X_X = block_diag(*[K_X_X]*D)
        K_Xt_X = self.K(X_test, self.X_train)
        K_Xt_X = block_diag(*[K_Xt_X]*D)
        K_X_Xt = K_Xt_X.T
        K_Xt_Xt = self.K(X_test, X_test); 
        K_Xt_Xt = block_diag(*[K_Xt_Xt]*D)

        Sigma = self.sigma_n * np.identity(D)
        I = np.identity(len(self.X_train))
        y_concat = self.Y_train.T.stack(level=-1).values

        # predictive mean, covariance, variance
        
        mean = K_Xt_X @ np.linalg.inv( K_X_X + np.kron(Sigma,I) ) @ y_concat
        cov = K_Xt_Xt - K_Xt_X @ np.linalg.inv( K_X_X + np.kron(Sigma,I) ) @ K_X_Xt
        var = np.diag(cov)

        self.predictive_distribution = {'mean': mean, 'cov':cov, 'var':var}
        self.temp = [C, K_X_X, K_X_Xt, K_Xt_X, K_Xt_Xt, Sigma, I, y_concat]

        return pd.DataFrame(mean.reshape(D, -1)).T


In [36]:
model = GaussianProcessRegression(X_train, Y_train, GaussianKernel())
pred = model.predict(X_test)

mse = mean_squared_error(pred.values, Y_test.values)
print(mse)

C = model.temp[0]
K_X_X = model.temp[1]
K_X_Xt = model.temp[2]
K_Xt_X = model.temp[3]
K_Xt_Xt = model.temp[4]
Sigma = model.temp[5]
I = model.temp[6]
y_concat = model.temp[7]


MemoryError: Unable to allocate 7.43 GiB for an array with shape (31587, 31587) and data type float64

In [ ]:
# np.kron # 380.8650009942557
# block_diag(*[A]*N) # 380.8650009942557
# kron_N_A # 382.4756739107164
# for loop # 382.83785471791873